### Get whisper to `dmx.compiler` with alternative `Conv1d` subgraph (scatter implementation)

In [5]:
model_id = "openai/whisper-tiny"

In [ ]:
# original inference

import torch
from transformers import pipeline

pipe = pipeline(
    task="automatic-speech-recognition", 
    model=model_id,
    device="cuda",
)

pipe("audio.mp3")

In [ ]:
# add these additional lines before transformation/configuration to map Conv1d to our alternative version
from dmx.compressor.modeling.nn import experimental
from dmx.compressor.fx.transformer.utils import dmx_aware_mapping

dmx_aware_mapping["torch.nn.modules.conv.Conv1d"] = experimental.Conv1dScatter

# ---
# transformation and configuration
from dmx.compressor import DmxModel

pipe.model = DmxModel.from_torch(pipe.model)
pipe.model.to_basic_mode()

In [ ]:
with torch.no_grad():
    pipe("audio.mp3")

In [ ]:
pipe.model.dmx_config

In [ ]:
dmx_aware_mapping

```pipe.model._gms``` is a dictionary that contains all GraphModules created so far, with their input signature reprensentation as keys

In [ ]:
pipe.model._gms

The below inputs has the same signature as the first forward pass in generation pipe, hence we already have the corresponding gm and it will be reused

In [11]:
# do a forward that triggers JIT tracing through encoder
inputs = {
    "input_features": torch.rand(1, 80, 3000).to("cuda"),
    "decoder_input_ids": torch.tensor([[50258]], device="cuda"),
}
output = pipe.model(**inputs)

```pipe.model.make_compiler_graphs()``` is a function to go through all gms in ```model._gms``` and turn them to compiler graphs, which will be stored in ```pipe.model._compiler_graphs```

In [12]:
import dmx.ops
pipe.model.make_compiler_graphs()

In [ ]:
pipe.model._compiler_graphs

The first gm contains the encoder, which contains the Conv1ds

In [14]:
key = list(pipe.model._gms.keys())[0]

In [ ]:
pipe.model._gms[key].graph.print_tabular()

In [ ]:
pipe.model._compiler_graphs[key].graph.print_tabular()